In [45]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, unix_timestamp, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

#### Question 1: Spark version

In [46]:
spark.version

'3.5.4'

#### Question 2: repartition

In [48]:
df_yellow = spark.read.parquet("data/raw/yellow_tripdata_2024-10.parquet")
df_yellow = df_yellow.repartition(4)
df_yellow.write.parquet("yellow_trips/2024/10/", mode="overwrite")

#### Question 3: 15th october trips

In [49]:
df_filtered = df_yellow.filter(col("tpep_pickup_datetime").between("2024-10-15", "2024-10-16"))
df_filtered.count()

128895

#### Question 4: longest trip

In [51]:
df_yellow_duration = df_yellow.withColumn("duration", 
                   (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600)

df_yellow_duration.orderBy(col("duration").desc()).limit(1).select(col("duration")).show()


[Stage 73:=============================================>          (13 + 3) / 16]

+------------------+
|          duration|
+------------------+
|162.61777777777777|
+------------------+



#### Question 6: Least frequent pickup zone

In [53]:
schema = StructType([
    StructField("LocationID", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Zone", StringType()),
    StructField("service_zone", StringType()),
])

df_zones = spark.read.csv("data/raw/taxi_zone_lookup.csv", header=True, schema=schema)

In [54]:
df_zones.createOrReplaceTempView("zones")
df_yellow.createOrReplaceTempView("yellow_trips")

spark.sql("""
SELECT 
    zones.Zone,
    count(1) as trips
FROM yellow_trips
INNER JOIN zones ON zones.LocationID = yellow_trips.PULocationID
GROUP BY
    zones.Zone
ORDER BY
    trips ASC
""").show()

+--------------------+-----+
|                Zone|trips|
+--------------------+-----+
|Governor's Island...|    1|
|       Rikers Island|    2|
|       Arden Heights|    2|
|         Jamaica Bay|    3|
| Green-Wood Cemetery|    3|
|Charleston/Totten...|    4|
|   Rossville/Woodrow|    4|
|       Port Richmond|    4|
|Eltingville/Annad...|    4|
|       West Brighton|    4|
|         Great Kills|    6|
|        Crotona Park|    6|
|Heartland Village...|    7|
|     Mariners Harbor|    7|
|Saint George/New ...|    9|
|             Oakwood|    9|
|       Broad Channel|   10|
|New Dorp/Midland ...|   10|
|         Westerleigh|   12|
|     Pelham Bay Park|   12|
+--------------------+-----+
only showing top 20 rows

